# Geospatial Visualization and Analysis

Greg Lee
<br>
08.24.20

Note: If you run this notebook through vscode, you may need to adjust the python interpreter

In [1]:
#Libraries 

#Core
import pandas as pd
import matplotlib.pyplot as plt

#Geography Specific
import geopandas as gpd 
import fiona
import contextily as ctx

## Loading in the Data

Data loading can be a real pain in the neck. Here I showcase how to import several common data file types. Please note, often you will need to know the epsg for mapping, so it is worth grabbing when pulling the data.

Data Sources:
1. Census track information: www2.census.gov/geo/tiger/ (shapefile)
2. Educational Attainment: https://livingatlas.arcgis.com/en/home/ (textfile)
3. Utah Ski Resorts: https://gis.utah.gov/data/recreation/ski-areas/ (geodatabase)


In [2]:
#Please set project directory pathway for your computer. This will be absolute for now.
PROJECT_DIR = "/home/boogie2/Hanson_Lab/mapping_tutorial/"

#Load the data:

#Shapefile Load
ut_census_tracts_raw = gpd.read_file(PROJECT_DIR + '/data/tl_2018_cs_shapefile')

#Textfile Data Load
educational_attainment_raw = pd.read_csv(PROJECT_DIR +'/data/ACS_Education_Attainment.txt')

#Geodatabase load: Geopandas does not play well- use fiona instead!
ski_resorts_raw =fiona.open(PROJECT_DIR + '/data/SkiAreaLocations.gdb')
ski_resort_df = gpd.GeoDataFrame.from_features([feature for feature in ski_resorts_raw], crs=ski_resorts_raw.crs)
# Get the order of the fields in the Fiona Collection; add geometry to the end
columns = list(ski_resorts_raw.meta["schema"]["properties"]) + ["geometry"]
# Re-order columns in the correct order
ski_resort_df = ski_resort_df[columns]

#Load Utah Counties too!
ut_counties_raw = gpd.read_file(PROJECT_DIR + '/data/utah_counties_shapefile')

/home/boogie2/anaconda3/envs/geo_mapping/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [3]:
#Lets look at the data: 

#These data are census tracts - useful later on!
ut_census_tracts_raw.head(2)

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,49,049,001001,49049001001,10.01,Census Tract 10.01,G5020,S,1693893,0,+40.2934782,-111.6812127,"POLYGON ((-111.69503 40.29713, -111.69326 40.2..."
1,49,049,001002,49049001002,10.02,Census Tract 10.02,G5020,S,1508099,0,+40.2861635,-111.6792437,"POLYGON ((-111.69180 40.28977, -111.68950 40.2..."


In [4]:
educational_attainment_raw.head(2) #This is our education data

,OBJECTID,GEOID,ALAND,AWATER,NAME,State,County,B15002_001E,B15002_001M,B15002_002E,...,B15002_calc_numAAE,B15002_calc_numAAM,B15002_calc_pctAAE,B15002_calc_pctAAM,B15002_calc_numGEBAE,B15002_calc_numGEBAM,B15002_calc_pctGEBAE,B15002_calc_pctGEBAM,Shape__Area,Shape__Length
0,21190,49025130100,9.665790e+09,305873544.0,Census Tract 1301,Utah,Kane County,1499,178.0,755,...,152,64,10.1,4.096205,244,69,16.3,4.177582,1.530665e+10,1.953153e+06
1,21191,49025130200,6.681224e+08,295663.0,Census Tract 1302,Utah,Kane County,3618,206.0,1824,...,303,128,8.4,3.505584,1155,238,31.9,6.322112,1.052911e+09,1.767308e+05


In [5]:
ski_resort_df.head(2) #This is our ski resort data

,Name,geometry
0,Park City - Canyons at Park City,POINT (453161.430 4503919.420)
1,Park City - Park City Mountain,POINT (457124.570 4500351.650)


In [6]:
ut_counties_raw.head(2)

,FID,COUNTYNBR,ENTITYNBR,ENTITYYR,NAME,FIPS,STATEPLANE,POP_LASTCE,POP_CURRES,GlobalID,FIPS_STR,COLOR4,SHAPE_Leng,SHAPE_Area,geometry
0,1,03,2.010031e+09,2010.0,CACHE,5.0,North,113307,128886,{4E8EDA26-0663-4E36-BA63-3FFB6E26F3E9},49005,2,3.071504,0.328661,"POLYGON ((-112.15617 41.99773, -112.15399 41.9..."
1,2,07,2.010071e+09,2010.0,DUCHESNE,13.0,Central,18721,20850,{C3E5EC81-5770-4B25-9DA0-85053A93877F},49013,4,3.966961,0.891751,"POLYGON ((-110.25174 40.83235, -110.25071 40.8..."


## Joining Data

As it stands, the current .txt file representing education is useless. We need it joined to either track or county. For ease in this analysis, we will just use the counties!

Note in the Educational Data: 
1. B15002_001E - Total Population 25 Years and Over
2. B15002_calc_numLTHSM - Population 25 Years and Over whose Highest Education Completed is Less Than High School - Margin of Error

These come from https://www.arcgis.com/home/item.html?id=84e3022a376e41feb4dd8addf25835a3&view=list&sortOrder=true&sortField=defaultFSOrder#data --> fields

In [7]:
#We can extract the County FIPS code from GEOID in educational_attainment_raw to merge with ut_counties_raw
educational_attainment_raw['COUNTY_FIPS'] = educational_attainment_raw['GEOID'].astype('str').str[3:5].astype('int')

#Need to make the data types both int
ut_counties_raw['COUNTY_FIPS'] = ut_counties_raw['FIPS'].astype('int')

In [8]:
#Merge with the County Data frame
ed_attain_county_gdf = ut_counties_raw.merge(educational_attainment_raw,how='right',on='COUNTY_FIPS')

In [9]:
#Let's first find the percentage of people in each tract who have not graduated from high school
ed_attain_county_gdf['pop_perc_not_hs_grad'] = ed_attain_county_gdf['B15002_calc_numLTHSE'] / ed_attain_county_gdf['B15002_001E']

#Now that we have values for all the counties let's group down, so we have average number 
#who did not graduate from hs
avg_edu_df = ed_attain_county_gdf.groupby(['COUNTY_FIPS']).agg({'geometry':'first',
                                                                'pop_perc_not_hs_grad':'mean'}).reset_index()

In [10]:
avg_edu_df.head(2)
#Sweet! These values make sense

,COUNTY_FIPS,geometry,pop_perc_not_hs_grad
0,1,"POLYGON ((-112.51541 38.57285, -112.51540 38.5...",0.093589
1,3,"POLYGON ((-113.47489 41.99331, -113.47326 41.9...",0.059367


In [11]:
#TO DO: 
#Get a track level geodataframe with educational data of a variable of your choice!

## Converting Projections

Each of the loaded databases has a different mapping! Most often, these databases will have an associated projections which can be accessed easily utilizing the `.crs` command. Note, with lots of lat/long data there is typically no projection - this utilizes epsg 4326.  

One annoying part of groupby's is they force the geodataframe to become a dataframe. We will first need to revert avg_edu_df to a geodataframe then alter the projections. 

In [12]:
type(avg_edu_df)

pandas.core.frame.DataFrame

In [13]:
#Converting to a geodataframe
avg_edu_gdf = gpd.GeoDataFrame(avg_edu_df, geometry=avg_edu_df['geometry'])

#Assign back the projection
avg_edu_gdf.crs = ed_attain_county_gdf.crs
print(type(avg_edu_gdf))

<class 'geopandas.geodataframe.GeoDataFrame'>


In [14]:
def projection_getter(dataframe_name,dataframe):
    print(dataframe_name + ' has the mapping: ' + str(dataframe.crs))

In [15]:
projection_getter("Average education by County data",avg_edu_gdf)
projection_getter("ski resorts data",ski_resort_df)

Average education by County data has the mapping: epsg:4326
ski resorts data has the mapping: +init=epsg:26912 +type=crs


## Reprojecting Data

We now need to convert all mappings to a unified projection. We will use 3857 since it is compatible with contextily - our background mapping program!

In [16]:
avg_edu_gdf = avg_edu_gdf.to_crs(epsg=3857)
ski_resort_gdf = ski_resort_df.to_crs(epsg=3857)

projection_getter("Average education by County data",avg_edu_gdf)
projection_getter("ski resorts data",ski_resort_gdf)

Average education by County data has the mapping: epsg:3857
ski resorts data has the mapping: epsg:3857


## Visualizing the Data

This is a basic view overview of plotting

In [17]:
fig,ax = plt.subplots(1,1,figsize=(20,20))
avg_edu_gdf.plot(column = 'pop_perc_not_hs_grad',
                    ax = ax,
                    alpha=0.8,
                    legend=True,
                    legend_kwds={'label': "% Population 25+ who did not graduate from hs"})

ax.set_title('% of Utah Population Without HS Degree')
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)

ctx.add_basemap(ax)

plt.close()
#fig.saveas('fig_name.png')

In [18]:
#Let's try taking only the subsample of wasatch counties
wasatchfront_countyfps=[49,35,11,29,57]
avg_edu_wasatch_gdf = avg_edu_gdf[avg_edu_gdf.COUNTY_FIPS.isin(wasatchfront_countyfps)]

fig,ax = plt.subplots(1,1,figsize=(20,20))
avg_edu_wasatch_gdf.plot(column = 'pop_perc_not_hs_grad',
                    ax = ax,
                    alpha=0.8,
                    legend=True,
                    legend_kwds={'label': "% Population 25+ who did not graduate from hs"})

ax.set_title('% of Wasatch Population Without HS Degree')
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)

ctx.add_basemap(ax)

plt.close()
#fig.saveas('fig_name.png')

In [19]:
#Let's Visualize some point data
fig,ax = plt.subplots(1,1,figsize=(20,20))
ski_resort_gdf.plot(ax = ax,
                    alpha=0.8,
                   markersize=80,
                    marker='o', 
                    color='red')

ax.legend(['Ski Resorts in Utah'])
ax.set_title('Locations of Ski Resorts')
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)

ctx.add_basemap(ax)

plt.close()
#fig.saveas('fig_name.png')

In [20]:
type(ut_counties_raw.to_crs(epsg=3857))

geopandas.geodataframe.GeoDataFrame

In [25]:
#Let's visualize a spatial join top see how many ski resorts there are per county
temp_join = gpd.sjoin(ut_counties_raw.to_crs(epsg=3857),ski_resort_gdf) #Spatial join Points to polygons
county_ski = dfsjoin.groupby('FIPS').agg({'geometry':'first','Name':'count'}).reset_index()

county_ski_gdf = gpd.GeoDataFrame(county_ski, geometry=county_ski['geometry'])

#TO DO: Finish plotting this to visualize which county has the most ski resorts (spoilers it isn't in the south!)